The idea is to put only important algorithms to Review

# DIFFERENCE ARRAY (LEET 2381)

Da usare quando si vuole fare un range update, invece che aggiornare ogni elemento da index 2 a 5 ad esempio. Incremento l’elemento all’indice 2 di + tot e tolgo all’indice end +1 – tot quindi se voglio aggiungere 2 a tutti gli elementi tra 2 e 5 devo fare

Start = [1, 2, 1, 1, 3, 0, 0]

Compute gradient array: array delle somme parziali, cioè prendo solo le differenze tra elementi

Arra_diff= [1, 1, -1, 0, 2, -3, 0, 0]   

last element is dummy. Il dummy serve se voglio aggiornare un range che ha dentro anche l’ultimo index dell’array

Aggiungo il valore a index 2 e tolgo a index 5+1:

Arra_diff= [1, 1, 1, 0, 2, -5, 0, 0]

Poi per ottenere la somma finale devo fare la somma cumulativa, l’ultimo elemento dell’array sum è dummy

Tot_sum = [1, 2, 3, 3, 5, 0,  0, 0]

Final_res = [1, 2, 3, 3, 5, 0,  0] #remove last element

In [ ]:
class Solution:
    def shiftingLetters(self, s: str, shifts: List[List[int]]) -> str:
        #0 sta dov'è, -num vai indietro e +num vai inavanti
        arr_diff = [0] * (len(s)+1)
        final_shifts = []
        for shift in shifts:
            if shift[2]:
                arr_diff[shift[0]] += 1
                arr_diff[shift[1]+1] -= 1
            else:
                arr_diff[shift[0]] -= 1
                arr_diff[shift[1]+1] += 1

        shift = 0
        for i in range(len(arr_diff)-1):
            shift += arr_diff[i]
            final_shifts.append(shift)

        res = ""
        for i, el in enumerate(final_shifts):
            new_char = chr((ord(s[i]) - ord('a') + el) % 26 + ord('a'))
            res += new_char
        return res


 # BIT (LEET 307) MUTABLE QUERY RANGE
 It is possible to use also segment tree tree

https://www.youtube.com/watch?v=uSFzHCZ4E-8 (BIT explanation)

 https://cp-algorithms.com/data_structures/fenwick.html   (binary)

 https://cp-algorithms.com/data_structures/segment_tree.html  (segment)

 We need tree structure because there is an updated otherwise a sum array would be suffienct e.g.:
 start = [ 5, 2, 9, -3, 5]

 summ array = [5, 7, 16, 13, 18]

 sum from index 1 to 3: summ_array[3] - sum_array[0] = 13-5 = 8

 So basically to get the sum from i to j, you need to take arr[j] - arr[i-1]
 but it doesnt work with updates (update in O(N) time) so we need to use a tree structure.

-------------------------------------------------------------------
BIT

 The magic of index & -index lies in efficiently isolating and leveraging the lowest set bit:

Addition (+=): Move to the next index that overlaps the current range.
Subtraction (-=): Move to the parent index, narrowing the range.

use addition for updates and subtraction for summing, to get the parent node

Intuition Behind the Difference
Updates (+= index & -index): Propagate changes forward to larger indices that depend on the current index.
Queries (-= index & -index): Aggregate contributions backward from smaller indices that form the prefix sum.







In [ ]:
class BIT:
      """
    Implementation of Binary Indexed Tree/Fenwick Tree

    Memory:
        creation - O(n)
        update   - O(1)
        get_sum  - O(1)

    Time:
        creation - O(n*log(n))
        update   - O(log(n))
        get_sum  - O(log(n))
    """
    def __init__(self, nums: List[int]):
        self.size = len(nums)
        self.bit = [0] * (self.size +1)
        for i in range(self.size):
            self.update(i+1, nums[i])

    def update(self, index: int, delta: int) -> None:
        while index <= self.size:
            self.bit[index] += delta
            index += index & -index #set LSB

    def query(self, index) -> int:
        res = 0
        while index > 0:
            res += self.bit[index]
            index -= index & -index
        return res

class NumArray:
    """
    Time:   O(N∗Log(N))
    Memory: O(n)
    """
    def __init__(self, nums: List[int]):
        self.nums = nums
        self.bit = BIT(nums)

    def update(self, index: int, val: int) -> None:
        self.bit.update(index + 1, val - self.nums[index])
        self.nums[index] = val

    def sumRange(self, left: int, right: int) -> int:
        return self.bit.query(right+1) - self.bit.query(left)


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# obj.update(index,val)
# param_2 = obj.sumRange(left,right)

# IMMUTABLE QUERY RANGE (LEET 303)

non serve la tree structure perchè non abbiamo updates
quindi basta utilizzare l'array della prefix sum, cioè somma cumulativa e fare somm:  arr[right] - arr[left-1]

In [ ]:
class NumArray:

    def __init__(self, nums: List[int]):
        self.sum = [0] * (len(nums)+1)

        #somma cumulativa
        for i in range(len(nums)):
            self.sum[i+1] = self.sum[i] + nums[i]

    #ho spostato gli indici perchè ho fatto la sum di len n +1 così risulta + facile fare left-1, quindi left-1 +1 = left e right +1 = right+1
    #così posso riferirmi all'array di sum facilmente
    def sumRange(self, left: int, right: int) -> int:
        return self.sum[right+1] - self.sum[left]


# Your NumArray object will be instantiated and called as such:
# obj = NumArray(nums)
# param_1 = obj.sumRange(left,right)

# PREFIX SUM & SUFFIX SUM (LEET 1769)

In questo caso farlo senza prefix e suffix sarebbe n^2. Quello che faccio con prexis sum e suffix sum è calcolarsi prima la somma da sx a dx e poi da dx a sx, il risultato finale è dato dalla somma di prefix sum e suffix sum.

In questo caso nel leet si poteva ottenere prefix sum e suffix sum tenendosi il numero di biglie incontrate fino a quel punto e aggiornandolo in caso si trovasse una nuova biglia, e lo si aggiungeva al risultato corrente in modo che il numero totale di biglie da muovere venisse aggiornato e quindi anche il numero totale di movimenti.

In [ ]:
class Solution:
    def minOperations(self, boxes: str) -> List[int]:
        n = len(boxes)
        #computo da sx a dx e da dx a sx così posso farlo in tempo n,
        #ogni volta aggiungo il numero di palle perchè per andare da una posizione ad una adiacente mi basta
        #computare il numero di palle e aggiungerle perchè devo spostare ogni volta tot palle per inserle nel box i
        l_r = [0] * n
        r_l = [0] * n

        tot_balls = 0
        tot_operations = 0
        for i in range(n):
            tot_operations += tot_balls
            l_r[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        tot_balls = 0
        tot_operations = 0
        for i in reversed(range(n)):
            tot_operations += tot_balls
            r_l[i] = tot_operations
            if boxes[i] == '1':
                tot_balls += 1

        for i in range(n):
            l_r[i] += r_l[i]
        return l_r

# GRAPH SHORTEST PATH (LEET 1368)

if weights are 0-1 in a directed graph you can use BFS 0-1 that means if weight is 1 append it to the right if it 0 have priority so append it to left.
Otherwise use heapqueue

In [ ]:
class Solution:
    def minCost(self, grid: List[List[int]]) -> int:
        directions = [(0,1,1), (0,-1,2), (1,0,3), (-1,0,4)]
        queue = deque([(0,0,0)]) # row, col , path
        visited = set()
        rows = len(grid)
        cols = len(grid[0])

        while queue:
            r, c, path = queue.popleft()
            if (r,c) == (rows-1, cols-1):
                return path
            if (r,c) in visited:
                continue
            visited.add((r,c))

            for x,y,d in directions:
                if 0 <= x + r < rows and 0 <= y + c < cols and (r+x, c+y):
                    if d == grid[r][c]:
                        queue.appendleft((r+x, c+y, path))
                    else:
                        queue.append((r+x, c+y, path + 1))

# Multi-source BFS (Leet 407)

graph traversal problem.

The idea is to add the border to the min heap bcs no water can be trapped by the border, and as we expand our search through bfs guided by mean heap, we can find adiacent cells that might be contained and so the curr maximum heigh will be lower than the height of the found cell. This way we know that we can trap water on tat cell. if the cell was visited before with an adiacent cell of lower height of that it means that no water can be trapped on that cell as it would flow through the lower height cells.

When i arrive at popping a cell i know the max height currently and i get water to trap only by doing the max height - curr height that is positive only if there is water to trap otherwise is gonna be 0 because we are visitng in order of height


The hard part is to understand the intuition and that res += maxheight - h is always optimal meaning that it always stores the trapped water because the other adjacent cells if of lower height or at the border are already visited

In [ ]:
class Solution:
    def trapRainWater(self, heightMap: List[List[int]]) -> int:
        heap = []
        ROWS, COLS = len(heightMap), len(heightMap[0])
        visited = set()
        DIRECTIONS = [(0,1), (1,0), (-1,0), (0,-1)]
        # put all border cells to the heap, no water can be contained by them so makes sence to explore them first
        for row in range(ROWS):
            for col in range(COLS):
                if row in [0, ROWS-1] or col in [0, COLS-1]:
                    heapq.heappush(heap, (heightMap[row][col], row, col))
                    visited.add((row, col))
        
        #multi-source BFS
        maxheight = 0
        res = 0
        while heap:
            h, r, c = heapq.heappop(heap)
            maxheight = max(maxheight, h)
            res += maxheight - h

            for d in DIRECTIONS:
                x, y = d
                new_r, new_c = r+x, c+y
                if (new_r, new_c) in visited:
                    continue
                if 0 <= new_r < ROWS and 0 <= new_c < COLS :
                    heapq.heappush(heap, (heightMap[new_r][new_c], new_r, new_c))
                    visited.add((new_r, new_c))


        return res


# Smart use of dictionaries to takle O(1) operations (LEET 2661)

You are given a 0-indexed integer array arr, and an m x n integer matrix mat. arr and mat both contain all the integers in the range [1, m * n].
Go through each index i in arr starting from index 0 and paint the cell in mat containing the integer arr[i].
Return the smallest index i at which either a row or a column will be completely painted in mat.

If not useing the nums array i couldnt get back the r, c and if not using sums_rows, and sums_cols i couldnt compute the sum in O(1) time

In [ ]:
class Solution:
    def firstCompleteIndex(self, arr: List[int], mat: List[List[int]]) -> int:
        m = len(mat)
        n = len(mat[0])
        #dic from num to row, col
        nums = {}
        #another dic of row and col sums to determine the first painted?
        #{0 ..m)} for rows and {0...n} for cols
        sums_rows = defaultdict(int)
        sums_cols = defaultdict(int)
        for i in range(m):
            for j in range(n):
                nums[mat[i][j]] = (i,j)
                sums_rows[i] += mat[i][j]
                sums_cols[j] += mat[i][j]


        for i, el in enumerate(arr):
            r, c = nums[el]
            sums_rows[r] -= el
            sums_cols[c] -= el

            if sums_rows[r] == 0 or sums_cols[c] == 0:
                return i


# DP + sliding window (leet 2017)
My intuition:
2nd robot (goes second, so the function is innested) wants to maxize what is left from either top or bottom after the pass of 1st robot, so MAX(TOP, BOTTOM).
the first robot (goes first so outer function) want to minimize the result of the second robot so MIN(MAX(TOP, BOTTOM)).
So this problem is basically solved, I just need to compute the remaining top and bottom and that's it!
So how do i do that TOP: compute the sum of the first row and as you go you decrease values.
Remaining of the BOTTOM: as you go you sum what you found on the same col, bcs since you stayed on top, the top row value goes to 0 aka difference. and the value on bottom row remains for the second robot so sum to bottom value. that's it! 





Solution:
Step 1: Sum the Top Row

Calculate the total sum of the top row (grid[0]).
This represents the score available to the second robot  initially if the first robot doesn’t take any cells from the top row.
Why? The top row sum decreases as the first robot progresses.
Step 2: Traverse Columns 

Use a sliding window approach :
Deduct the value of the current column from the top row's sum (representing the first robot collecting these cells).
Add the value of the current column to the bottom row's cumulative sum (bottom).
Calculate the maximum score left for the second robot using max(top, bottom).
Track the minimum "maximum score" possible using res.
Step 3: Minimize the Maximum 

The goal is to minimize the "worst-case scenario" for the second robot, ensuring a balanced outcome.

In [ ]:
class Solution:
    def gridGame(self, grid):
        top = sum(grid[0])
        bottom = 0
        res = float('inf')
        
        for i in range(len(grid[0])):
            top -= grid[0][i]
            res = min(res, max(top, bottom))
            bottom += grid[1][i]
        
        return res

# Multi-source BFS (leet 1765) same principle of leet 407

source = water
then update the adjacent cells by curr cell +1

In [ ]:
class Solution:
    def highestPeak(self, isWater: List[List[int]]) -> List[List[int]]:
        m = len(isWater)
        n = len(isWater[0])

        grid = [[float("inf")] * n for _ in range(m)]
        directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
        visited = set()
        queue = deque()
        #add water
        for row in range(m):
            for col in range(n):
                if isWater[row][col]:
                    grid[row][col] = 0
                    queue.append((row, col))
                    visited.add((row, col))
        
        while queue:
            x, y = queue.popleft()
            for d in directions:
                new_x = x + d[0]
                new_y = y + d[1]
                if 0 <= new_x < m and 0 <= new_y < n and (new_x, new_y) not in visited:
                    grid[new_x][new_y] = grid[x][y] + 1
                    queue.append((new_x, new_y))
                    visited.add((new_x, new_y))

        return grid


# LEET 1267

the trick is to sum cols and rows before counting the connected servers since one serve can be counted only once

In [ ]:
class Solution:
    def countServers(self, grid: List[List[int]]) -> int:
        res = 0
        m = len(grid)
        n = len(grid[0])

        
        #sum rows, sum cols
        sum_rows = {}
        sum_cols = {}

        for i in range(m):
            sum_rows[i] = sum(grid[i])
        for j in range(n):
            col = 0
            for i in range(m):
                col += grid[i][j]
            sum_cols[j] = col

        for i in range(m):
            for j in range(n):
                if (sum_rows[i] > 1 or sum_cols[j] > 1 )and grid[i][j]==1:
                    res+=1
                
        return res

# BFS graph traversal and in degree (Leet 802)

The idea is to reverse the edges and start from the terminal nodes, then for each node found add it to result and queue only if the in_degree is 0, a.k.a. have paths only from the terminal nodes. 
This is possible only if you reverse the edge list

In [1]:
class Solution:
    def eventualSafeNodes(self, graph: List[List[int]]) -> List[int]:
        #io farei  edges al contrario, quindi parto dai nodi terminali, vedo dove finiscono, se finiscono in nodo che ha come entrata un path da un nodo terminale e basta allora viene aggiunto anche quello alla risposta, altrimenti no. così dovrebbe bastare una sola visita del grafo invece che fare una visita per nodo.
        #use in-degree to find how many edges are going in


        reversed_adj_list = defaultdict(list)
        in_degree = {}
        res = []
        queue = deque()
        n = len(graph)
        for i in range(n):
            for el in graph[i]:
                reversed_adj_list[el].append(i) 
            if len(graph[i]) == 0:
                queue.append(i)
                in_degree[i] = 0
            else:
                in_degree[i] = len(graph[i])

        while queue:
            el = queue.popleft()
            res.append(el)
            for neighbour in reversed_adj_list[el]:
                in_degree[neighbour] -= 1
                if in_degree[neighbour] == 0:
                    queue.append(neighbour)

        res.sort()
        return res



NameError: name 'List' is not defined